# Keyword Extraction

This script is more or less an Extra as most of it is not used in our Process.

This is Rake. Rake is a small library that takes some integrated stopword-lists and extracts keywords of sentences.
We use Rake for keyword extraction of our user input as we do only use keywords for symptom prediction in our intents anyway.
Furthermore we do not need anything else than the keywords themself. We neither need any particular medical keywords as normal people decribe their symptoms with commonly used words. We also get only one short sentence from the user which makes any entitiy recognition process useless.

In [1]:
import RAKE
Rake = RAKE.Rake(RAKE.SmartStopList())
#Rake = RAKE.Rake(RAKE.NLTKStopList())
Rake.run("I have to sneeze very badly. My head hurts and is in pain.")

[('head hurts', 4.0), ('sneeze', 1.0), ('badly', 1.0), ('pain', 1.0)]

### Extra

The rest of this notebook is not used and is copied from a website for Entity Recognition which would be usable if we would extend our model to not only guess one symptom at a time but many with the user giving more sentences or words at once.

In [2]:
from nltk import tokenize
from operator import itemgetter
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [3]:
doc = "I have a headache. My belly hurts and my ears are swollen."

In [4]:
stop_words = set(stopwords.words('english'))

In [5]:
total_words = doc.split()
total_word_length = len(total_words)
print(total_word_length)

12


In [6]:
total_sentences = tokenize.sent_tokenize(doc)
total_sent_len = len(total_sentences)
print(total_sent_len)

2


In [7]:
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
print(tf_score)

{'I': 0.08333333333333333, 'headache': 0.08333333333333333, 'My': 0.08333333333333333, 'belly': 0.08333333333333333, 'hurts': 0.08333333333333333, 'ears': 0.08333333333333333, 'swollen': 0.08333333333333333}


In [8]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

In [9]:
idf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

print(idf_score)

{'I': 0.6931471805599453, 'headache': 0.6931471805599453, 'My': 0.6931471805599453, 'belly': 0.6931471805599453, 'hurts': 0.6931471805599453, 'ears': 0.6931471805599453, 'swollen': 0.6931471805599453}


In [10]:
tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
print(tf_idf_score)

{'I': 0.057762265046662105, 'headache': 0.057762265046662105, 'My': 0.057762265046662105, 'belly': 0.057762265046662105, 'hurts': 0.057762265046662105, 'ears': 0.057762265046662105, 'swollen': 0.057762265046662105}


In [11]:
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

In [12]:
print(get_top_n(tf_idf_score, 5))

{'I': 0.057762265046662105, 'headache': 0.057762265046662105, 'My': 0.057762265046662105, 'belly': 0.057762265046662105, 'hurts': 0.057762265046662105}


Note here that words like 'I' and 'My' get recognized as well here which would not be useful in our case.